In [ ]:
import re, json, ast
from typing import Tuple, Any, Optional

# ── regexes ────────────────────────────────────────────────────────────────────
_THINK_BLOCK = re.compile(r"<think>.*?</think>", re.DOTALL)
_JSON_FENCE = re.compile(r"```json\s*(.*?)\s*```", re.DOTALL)
_FIRST_BRACE_LIST = re.compile(r"(\{.*?\}|\[.*?\])", re.DOTALL)
_COMMENT_LINE = re.compile(r"//.*?(?=\n|$)")
_TRAILING_COMMA = re.compile(r",(\s*[\]}])")

# ── helpers ────────────────────────────────────────────────────────────────────
def _strip_comments_and_commas(txt: str) -> str:
    """Remove //-style comments and any comma that comes *right* before ] or }."""
    txt = _COMMENT_LINE.sub("", txt)
    txt = _TRAILING_COMMA.sub(r"\1", txt)
    return txt

# ── main entry point ───────────────────────────────────────────────────────────
def parse_agent_reply(raw: str) -> Tuple[str, Optional[Any]]:
    """
    Return (message_without_thinks, parsed_json_or_None).
    """
    # 1Drop every <think>…</think>
    message = _THINK_BLOCK.sub("", raw).strip()

    # Locate JSON text
    m = _JSON_FENCE.search(message)
    json_text = m.group(1) if m else None
    if json_text is None:
        m2 = _FIRST_BRACE_LIST.search(message)
        json_text = m2.group(1) if m2 else None

    # Parse, tolerating comments & trailing commas
    data: Optional[Any] = None
    if json_text:
        cleaned = _strip_comments_and_commas(json_text)
        try:
            data = json.loads(cleaned)
        except json.JSONDecodeError:
            try:
                data = ast.literal_eval(cleaned)
            except Exception:
                pass

    return message, data


In [3]:
str= """<think>
Okay, let's tackle this user's request. They provided a fire scenario with specific geographic coordinates and are asking for local SOPs and drone waypoints to contain the fire.
</think>

<think>
Putting this all together: first call assess_fire_danger, then retrieve local SOPs, then generate safe waypoints based on the danger assessment and POI locations.
</think>

<think>
yooooo
</think>

### 🔥 Local SOPs for High Fire Region (Based on Fire Danger Assessment)

### 🚁 Drone Waypoints for Fire Containment (6-12 Safe Points)

**Objective:** Create a perimeter of safe waypoints to monitor, suppress, and isolate the fire using drones while avoiding high-risk subgrids.

#### ✅ **Waypoints (6-12 Points):**  
**Ensure these points are in low-risk zones and avoid high-risk subgrids.**

```json
[
  [47.742888, 7.959087], //SDSDSD
  [47.743009, 7.959254], // SDSDS
  [47.743675, 7.993508],
  [47.744334, 7.969179],
]
```
---

### 🚨 **Safety Notes:**
- **Avoid subgrids 5, 6, 11:** These have higher fire danger scores (≥ 45).
- **Use drones near shelters** (e.g., Zellermooshütte, Tiefensohlenhütte) only if confirmed safe by fire danger assessment.
- **Plan for real-time adjustments** based on fire spread patterns (e.g., wind direction, vegetation type)."""

In [4]:
clean_msg, path_list = parse_agent_reply(str)
print(f"Message: {clean_msg}")
print("MESSAGE OVER")
print(f"Path List: {path_list}")

Message: ### 🔥 Local SOPs for High Fire Region (Based on Fire Danger Assessment)

### 🚁 Drone Waypoints for Fire Containment (6-12 Safe Points)

**Objective:** Create a perimeter of safe waypoints to monitor, suppress, and isolate the fire using drones while avoiding high-risk subgrids.

#### ✅ **Waypoints (6-12 Points):**  
**Ensure these points are in low-risk zones and avoid high-risk subgrids.**

```json
[
  [47.742888, 7.959087], //SDSDSD
  [47.743009, 7.959254], // SDSDS
  [47.743675, 7.993508],
  [47.744334, 7.969179],
]
```
---

### 🚨 **Safety Notes:**
- **Avoid subgrids 5, 6, 11:** These have higher fire danger scores (≥ 45).
- **Use drones near shelters** (e.g., Zellermooshütte, Tiefensohlenhütte) only if confirmed safe by fire danger assessment.
- **Plan for real-time adjustments** based on fire spread patterns (e.g., wind direction, vegetation type).
MESSAGE OVER
Path List: [[47.742888, 7.959087], [47.743009, 7.959254], [47.743675, 7.993508], [47.744334, 7.969179]]


In [4]:
print(f"Path List: {type(path_list)}")

Path List: <class 'list'>
